In [1]:
# streamlit_app.py

import pandas as pd
import json
import numpy as np
import datetime as dt
import streamlit as st
from plotnine import *
from plotnine.data import mtcars
from supabase import create_client, Client

# Initialize connection.
# Uses st.cache_resource to only run once.
@st.cache_resource
def init_connection():
    url = st.secrets["supabase_url"]
    key = st.secrets["supabase_key"]
    return create_client(url, key)

supabase = init_connection()

# Perform query.
# Uses st.cache_data to only rerun when the query changes or after 10 min.
# @st.cache_data(ttl=60)
def run_query(tbl_name):
    return supabase.table(tbl_name).select("*").execute()

# get data and format as DF
rows = run_query("bookings")
df = pd.DataFrame(rows.data)

# setup helper tables to catch available places to book
rooms = ["113", "115", "117"]
places = [3, 3, 1]
rooms_places = pd.DataFrame({"roomno": rooms, "places": places})

# not sure where this is needed
time_slots = ["08:00", "09:00", "10:00", "11:00", "12:00", "13:00", 
              "14:00", "15:00", "16:00", "17:00", "18:00"]

df.head()

2023-04-13 12:36:18.401 
  command:

    streamlit run /Users/blacat01/.local/share/virtualenvs/offices-py-ei01VrcE/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


,id,created_at,name,roomno,date_start,date_end
0,27,2023-04-12T20:25:57.709351+00:00,toby\n\n\n,115,2023-04-13T08:00:00,2023-04-13T14:00:00
1,28,2023-04-12T20:26:21.400685+00:00,simon,117,2023-04-17T08:00:00,2023-04-17T16:00:00
2,29,2023-04-12T20:29:15.717595+00:00,lisa,117,2023-04-13T08:00:00,2023-04-13T17:00:00


In [13]:
# group by roomnumber
# convert date_start and date_end columns to datetime format
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

# grouped = df.groupby('roomno')

# reformat to longer
df_long = (df
           .melt(id_vars=['id', 'name', 'roomno'], value_vars=['date_start', 'date_end'], var_name='what', value_name='time'))


df_long = df_long.groupby('id')

# define function to create date range for each group
def create_date_range(group):
    start = group['date_start'].min()
    end = group['date_end'].max()
    return pd.date_range(start=start, end=end, freq='H')

# apply function to each group and concatenate results into a single DataFrame
hourly_df = pd.concat([pd.DataFrame({'roomno': name, 'hour': create_date_range(group)}) for name, group in grouped])

hourly_df.head(20)

# # define function to calculate occupancy rate for each hour within each group
# def calculate_occupancy_rate(group):
#     occupancy = pd.Series(0, index=pd.date_range(start=group['date_start'].min().floor('H'), end=group['date_end'].max().ceil('H'), freq='H'))
#     for idx, row in group.iterrows():
#         occupancy[row['date_start'].floor('H'):row['date_end'].ceil('H')] += 1
#     occupancy_rate = occupancy / len(group)
#     return occupancy_rate

# # apply function to each group and concatenate results into a single DataFrame
# occupancy_df = pd.concat([calculate_occupancy_rate(group) for name, group in grouped])

# print(occupancy_df)

,roomno,hour
0,115,2023-04-13 08:00:00
1,115,2023-04-13 09:00:00
2,115,2023-04-13 10:00:00
3,115,2023-04-13 11:00:00
4,115,2023-04-13 12:00:00
5,115,2023-04-13 13:00:00
6,115,2023-04-13 14:00:00
0,117,2023-04-13 08:00:00
1,117,2023-04-13 09:00:00
2,117,2023-04-13 10:00:00


In [3]:
# group by roomnumber
grouped = df.groupby('roomno')

# define function to create date range for each group
def create_date_range(group):
    start = group['date_start'].min()
    end = group['date_end'].max()
    return pd.date_range(start=start, end=end, freq='H')

# apply function to each group and concatenate results into a single DataFrame
hourly_df = pd.concat([pd.DataFrame({'roomno': name, 'hour': create_date_range(group)}) for name, group in grouped])

# keep only business hours
hourly_df = hourly_df[(hourly_df['hour'].dt.hour >= 8) & (hourly_df['hour'].dt.hour < 20)]

hourly_df = pd.merge(hourly_df, rooms_places, on='roomno', how='left')

hourly_df = hourly_df.groupby('roomno')

# calc occ count
# hourly_df['occupancy_count'] = hourly_df.apply(lambda x: ((x['hour'] >= pd.to_datetime(df['date_start'])) & (x['hour'] <= pd.to_datetime(df['date_end']))).sum(), axis=1)

# calculate the occupancy rate as the occupancy count divided by the total number of hours

# hourly_df['occupancy_rate'] = hourly_df['occupancy_count']/hourly_df['places']

hourly_df.head()

,roomno,hour,places
0,115,2023-04-13 08:00:00,3
1,115,2023-04-13 09:00:00,3
2,115,2023-04-13 10:00:00,3
3,115,2023-04-13 11:00:00,3
4,115,2023-04-13 12:00:00,3
7,117,2023-04-13 08:00:00,1
8,117,2023-04-13 09:00:00,1
9,117,2023-04-13 10:00:00,1
10,117,2023-04-13 11:00:00,1
11,117,2023-04-13 12:00:00,1
